# Dynamic Programming

This notebook assembles the advanced dynamic programing problem to helpe summerize the pattern of solution same familiar problem.

# Coin Change II

You are given coins of different denominations and a total amount of money. Write a function to compute the number of combinations that make up that amount. You may assume that you have infinite number of each kind of coin.

**Example 1:**
```py
Input: amount = 5, coins = [1, 2, 5]
Output: 4
Explanation: there are four ways to make up the amount:
5=5
5=2+2+1
5=2+1+1+1
5=1+1+1+1+1
```
**Example 2:**
```py
Input: amount = 3, coins = [2]
Output: 0
Explanation: the amount of 3 cannot be made up just with coins of 2.
```
**Example 3:**
```py
Input: amount = 10, coins = [10] 
Output: 1
```

Here, I would like to follow the steps be introduced on Dynamic programming I. When we think the problem can be solved the use the dynamic programming approach, the first thing we need to come up in mind is to determine the recursion relation of the subproblems. Sometimes, think the subproblem through the arguments can be a good start.

Compare this problem to the coin change I, the different is about our output, here we need to output the number of combinations that make up that amount. But our subproblem will have the same situation:

1. Use the first denomination of the coin, to decrease our amount and recursion our function with a new amount and same coin set.
2. Not use the first coin on the coins set, recursion our function with the new coins set and the same amount.

Our relation can be write as `F(coins, amount) = F(coins[1:], amount) + F(coins, amount-coins[0])`. Which `F(coins, amount)` means the number of combinations with the `coins` that can make up the `amount`.

In [1]:
def naive_make_change(amount, coins):
    # base case is the amount == 0 and coins set is empty
    if amount == 0:
        return 1
    if not coins:
        return 0
    
    if amount < coins[0]:
        # if amount is bigger than our the first denomination on our coins set,
        # we need to skip it as it can not help us to change our amount.
        return naive_make_change(amount, coins[1:])
    
    # when the first coin on our coins set can be use to change,
    # we have two choice to do:
    # 1. not use the first coin, skip it use the remain coins set to change our amount
    # 2. use the first coin, the amount decrease by its denomination. and for us can
    # use the first coin again, not change our coin sets.
    return naive_make_change(amount, coins[1:]) + naive_make_change(amount-coins[0], coins)

In [2]:
naive_make_change(5, [1, 2, 5])

4

In [3]:
import timeit
timeit.timeit('naive_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.025828660996921826

The sultion above is our recursion one, which will accumulate the call stack. Our time complexity : `O(2 ^ S+n)`. where S is the amount, n is denomination count. We can use cache to improve our time complexity. Our space complexity is `O(S+n)`.

In [4]:
def memo_make_change(amount, coins, memo={}):
    if amount == 0:
        return 1
    if not coins:
        return 0
    
    if amount < coins[0]:
        return memo_make_change(amount, coins[1:], memo)
    
    if (amount, coins[0]) in memo:
        return memo[(amount, coins[0])]
    ans = memo_make_change(amount, coins[1:], memo) + memo_make_change(amount-coins[0], coins, memo)
    memo[(amount, coins[0])] = ans
    return ans

In [5]:
memo_make_change(5, [1, 2, 5])

4

In [6]:
timeit.timeit('memo_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.0016042499992181547

So we can easy think of our subproblem from the arguments. Create a table for our subproblems, the row use the `coins set` argument, and the columns use our `ammount` argument. (I use the example 1 as our specifial condition for find the subproblem realtionship.

coins/amount | 0 | 1 | 2 | 3 | 4 | 5 
--|--|--|--|--|--|--
[]|1|0|0|0|0|0
[1]|1|1|1|1|1|1
[1,2]|1|1|2|2|3|3
[1,2,5]|1|1|2|2|3|4

We can notice that, our subproblem on the table can be difined as `table[coins_row][amount] = table[coins_row-1][amount] + table[coins_row][amount-c_i]`, where `c_i` means to the current coins_row's new added denomination. And if new added denomination is bigger thange the amount our `table[coins_row][amount] = table[coins_row-1][amount]`.

In [23]:
def bottom_up_make_change(amount, coins):
    # Creat the table, row is sub coins set. col is our amount.
    # Our row length will go to amount + 1, as we will conside the situration
    # that amount is 0, when no matter what our coins set going to be.
    # the our result will always be 1. Means change amount 0, there at least have 1
    # possible to get change.
    dp = [[1 if i == 0 else 0 for i in range(amount+1)] for _ in range(len(coins))]
    for a in range(1, amount+1):
        if coins[0] <= a:
            dp[0][a] = dp[0][a-coins[0]]
    
    for coin_row in range(1, len(coins)):
        for a in range(1, amount+1):
            if coins[coin_row] <= a:
                # Our coin can be use to change our amount `a`.
                dp[coin_row][a] = dp[coin_row-1][a] + dp[coin_row][a - coins[coin_row]]
            else:
                # Our current coin can not use to change our amount `a`.
                dp[coin_row][a] = dp[coin_row-1][a]
    return dp[-1][-1]

In [24]:
bottom_up_make_change(5, [1, 2, 5])

4

In [25]:
timeit.timeit('bottom_up_make_change(5, [1, 2, 5])', number=1000, globals=globals())

0.027406285000324715

Our time complexity is `O(n*s)` n is the number of denomination, s is amount. And the space complexity is `O(n*s)`. We can improve our space complexity. Thougth our subproblem table, is show us that the current rows is cacluted from the previous row of result. Which means we can improve it with a only row which can be `O(n)` space complexity.

In [34]:
def bottom_up_make_change_one_row(amount, coins):
    if amount == 0:
        return 1
    if not coins:
        return 0
    dp = [1 if i == 0 else 0 for i in range(amount+1)]
    for coin_id in range(len(coins)):
        for a in range(coins[coin_id], amount+1):
            dp[a] += dp[a - coins[coin_id]]
    return dp[-1]

In [35]:
bottom_up_make_change_one_row(5, [1, 2, 5])

4

In [36]:
timeit.timeit('bottom_up_make_change_one_row(5, [1, 2, 5])', number=1000, globals=globals())

0.013098011004331056

# Decode Way
A message containing letters from A-Z is being encoded to numbers using the following mapping:
```
'A' -> 1
'B' -> 2
...
'Z' -> 26
```
Given a non-empty string containing only digits, determine the total number of ways to decode it.

**Example 1:**
```py
Input: "12"
Output: 2
Explanation: It could be decoded as "AB" (1 2) or "L" (12).
```
**Example 2:**
```py
Input: "226"
Output: 3
Explanation: It could be decoded as "BZ" (2 26), "VF" (22 6), or "BBF" (2 2 6).
```
Our recursion conditions can be write as following.
1. Choice a single character to decode, and if it's zero we will return 0, as we can not decode string like `120`.
2. Choice two characters to decode, this time two characters need smaller than 26. As we can not decode string `36` to a single character.

In [41]:
def numDecodings(s):
    if not s:
        return 1
    if int(s[0]) == 0:
        return 0
    if len(s) > 1 and int(s[0:2]) <= 26:
        return numDecodings(s[1:]) + numDecodings(s[2:])
    
    else:
        return numDecodings(s[1:])

In [43]:
numDecodings("121")

3

Time complexity will be `O(2^n)` and space is `O(n)`.

Using memorization to decrease the time complexity to `O(n)`.

In [44]:
def memo_numDecodings(s, memo={}):
    if not s:
        return 1
    if int(s[0]) == 0:
        return 0
    if s in memo:
        return memo[s]
    if len(s) > 1 and int(s[0:2]) <= 26:
        ans = numDecodings(s[1:]) + numDecodings(s[2:])
        memo[s] = ans
        return ans
    else:
        ans = numDecodings(s[1:])
        memo[s] = ans
        return ans

In [45]:
memo_numDecodings("1201020112")

3

In [46]:
timeit.timeit('memo_numDecodings("1123123123")', number=1000, globals=globals())

0.001405659000738524

Our bottom up approach will be like, if we consider our sub-string is accumulate for the "" to `string` we will have the following table.
example `123`

sub-string|count
--|--|
""|1
"3"|1
"23"|1+1 = 2
"123"|2+1 = 3

We can define our recursion relation use `F(n)` which means the ways of decode the string from `n to len(string)-1`. `F(n) = F(n-1) + F(n-2)` when the two string can be smaller than `26`. Otherwise, `F(n) = F(n-1)`.

In [47]:
def bottom_up_numDecodings(s):
    if not s:
        return 1
    s = s[::-1]
    dp = [0 for _ in range(len(s)+1)]
    dp[0] = 1
    for i in range(1, len(s) + 1):
        if i == 1:
            dp[i] = 1 if s[i-1] != "0" else 0
        elif s[i-1] == "0":
            dp[i] = 0
        elif int(s[i-1] + s[i-2]) <= 26:
            dp[i] = dp[i-1] + dp[i-2]
        else:
            dp[i] = dp[i-1]
    return dp[-1]

In [48]:
bottom_up_numDecodings("1201020112")

3

In [49]:
timeit.timeit('bottom_up_numDecodings("1123123123")', number=1000, globals=globals())

0.023771116000716574

Our function `F(n)` only consider about the two previous value `F(n-1)` and `F(n-2)`, our function can use time complexity is `O(1)`.

In [50]:
def bottom_up_numDecodings_one_row(s):
    if not s:
        return 1
    # Initial the varible, is mean when the str(s)
    pre = 1
    cur = 0
    for i in range(len(s)-1, -1, -1):
        if i == len(s)-1:
            cur = 1 if s[i] != "0" else 0
        elif s[i] == "0":
            pre = cur
            cur = 0
        elif int(s[i] + s[i+1]) <= 26:
            pre, cur = cur, cur + pre
        else:
            pre = cur
    return cur

In [51]:
timeit.timeit('bottom_up_numDecodings_one_row("1123123123")', number=1000, globals=globals())

0.01660409799660556